# N-Queens MCMC P.2
## Estimating the number of solutions.

In [36]:
# Imports
import time, sys, timeit
from IPython.display import clear_output
import numpy as np
import itertools as it
import math
from tqdm.notebook import tqdm
import pickle

In [2]:
# For reproducibility
np.random.seed(2022)

In [3]:
# Known solutions
Z_known = [
2,
10,
4,
40,
92,
352,
724,
2_680,
14_200,
73_712,
365_596,
2_279_184,
14_772_512,
95_815_104,
666_090_624,
4_968_057_848,
39_029_188_884,
314_666_222_712,
2_691_008_701_644,
24_233_937_684_440,
227_514_171_973_736,
2_207_893_435_808_352,
22_317_699_616_364_044,
234_907_967_154_122_528]

In [4]:
def swap(z, i, j):
    """
    Swaps the elements of z at indices i and j, then returns z. Inplace.
    """
    z[[i, j]] = z[j], z[i]
    return z

In [5]:
def threats(z, i):
    """
    Returns number of queens threatening queen i.
    """
    Q = np.delete(np.arange(N), i) # Other queens
    return np.sum(abs(Q-i)==abs(z[Q]-z[i]))

In [6]:
def loss_diff(z, i, j):
    """
    Given a state z and swap operation (i,j), calculates the change in loss.
    """
    old = threats(z,i) + threats(z,j)
    y = swap(z, i, j)
    new = threats(y,i) + threats(y,j)
    z = swap(y, i, j)

    return new - old

In [7]:
### Loss function runs in n(n+1)/2 steps.
def loss(z):
    """
    Interprets z as chessboard with N queens threatening each other diagonally.
    Counts the number of unique pairs of threatening queens.
    """
    # Compute pairwise differences in z.
    row_diff = np.array([abs(z[j]-z[i]) for (i,j) in idx_pairs])
    loss = np.sum(col_diff==row_diff)
    return loss

In [18]:
# Find best betas for N = 4...27

# Initialisation
MAX_ITERS = 10000    # When to stop searching
REG_ITERS = 1000     # When to print
TRIALS    = 50       # How many times to run a pair (N, B)
Ns = np.arange(4,28) # Chessboard size
Bs = np.arange(1,10) # Betas to check
ConvDict = dict()    # Convergence time dictionary
for N in tqdm(Ns):
    
    C = math.comb(N,2)
    z0 = np.arange(1,N+1)
    idx_pairs = np.array(list(it.combinations(z0,2))) - [1,1]
    col_diff = np.array([j-i for (i,j) in idx_pairs])
    np.random.shuffle(z0)
    
    for B in tqdm(Bs):
        ConvIters = np.zeros(TRIALS) # Iterations till convergence
        for t in range(TRIALS):
            z = z0.copy()
            l = loss(z)
            for i in range(1, MAX_ITERS):

                # Choose a random swap
                m, n = idx_pairs[np.random.choice(C, size=1)][0]
                diff = loss_diff(z,m,n)
                acc = 1 if diff < 0 else np.exp(-B*diff)
                if np.random.rand() < acc:
                    z = swap(z,m,n)
                    l += diff

                # If a solution is found, exit.
                if l <= 0:
                    break
            ConvIters[t] = i
        ConvDict[(N,B)] = ConvIters

 78%|█████████████████████████████████████████████████████████████████▎                  | 7/9 [00:42<00:11,  5.99s/it]


 78%|█████████████████████████████████████████████████████████████████▎                  | 7/9 [02:16<00:21, 10.88s/it]


 78%|█████████████████████████████████████████████████████████████████▎                  | 7/9 [03:12<00:28, 14.07s/it]


100%|█████████████████████████████████████████████████████████████████████████████████| 23/23 [52:07<00:00, 135.96s/it]


In [38]:
ConvDict = {k: np.mean(v) for k, v in ConvDict.items()}
BestBetasDict = dict()
for k, v in ConvDict.items():
    if BestBetasDict.get(k[0]) == None:
        BestBetasDict[k[0]] = (k[1], v)
    else:
        BestBetasDict[k[0]] = (k[1], v) if (v < BestBetasDict[k[0]][1]) else BestBetasDict[k[0]]

In [41]:
BestBetas = np.array([[k, v[0]] for k, v in BestBetasDict.items()])
BestBetas

array([[ 4,  4],
       [ 5,  6],
       [ 6,  1],
       [ 7,  2],
       [ 8,  5],
       [ 9,  3],
       [10,  8],
       [11,  3],
       [12,  4],
       [13,  5],
       [14,  7],
       [15,  6],
       [16,  9],
       [17,  9],
       [18,  8],
       [19,  8],
       [20,  7],
       [21,  8],
       [22,  6],
       [23,  5],
       [24,  7],
       [25,  6],
       [26,  8]])

In [44]:
with open('avg_iters_NB.pkl', 'wb') as file:
    pickle.dump(ConvDict, file)

In [45]:
with open('betas.pkl', 'wb') as file:
    pickle.dump(BestBetas, file)

In [54]:
def divisors(n):
    # get factors and their counts
    factors = {}
    nn = n
    i = 2
    while i*i <= nn:
        while nn % i == 0:
            factors[i] = factors.get(i, 0) + 1
            nn //= i
        i += 1
    if nn > 1:
        factors[nn] = factors.get(nn, 0) + 1

    primes = list(factors.keys())

    # generates factors from primes[k:] subset
    def generate(k):
        if k == len(primes):
            yield 1
        else:
            rest = generate(k+1)
            prime = primes[k]
            for factor in rest:
                prime_to_i = 1
                # prime_to_i iterates prime**i values, i being all possible exponents
                for _ in range(factors[prime] + 1):
                    yield factor * prime_to_i
                    prime_to_i *= prime

    # in python3, `yield from generate(0)` would also work
    for factor in generate(0):
        yield factor
    
list(divisors(MAX_ITERS))

[1,
 2,
 4,
 8,
 16,
 5,
 10,
 20,
 40,
 80,
 25,
 50,
 100,
 200,
 400,
 125,
 250,
 500,
 1000,
 2000,
 625,
 1250,
 2500,
 5000,
 10000]

In [58]:
TM = np.array([[d, int(MAX_ITERS/d)] for d in list(divisors(MAX_ITERS))])

In [46]:
# Estimating the number of solutions.

# For every size chessboard with its best beta,
for NB in BestBetas:
    N, B = NB[0], NB[1]
    C = math.comb(N,2)
    
    # Fix the number of iterations (MAX_ITERATIONS) and find best balance of T and M
    for tm in TM:
        T, M = tm[0], tm[1]
        Bs = np.linspace(1, B, T)
        Z_ratios = []
        

4 4
5 6
6 1
7 2
8 5
9 3
10 8
11 3
12 4
13 5
14 7
15 6
16 9
17 9
18 8
19 8
20 7
21 8
22 6
23 5
24 7
25 6
26 8


In [330]:


    

    for t in range(T-1):

        # Print current length of L
        clear_output(wait = True)
        print(t)

        # Initial random sample
        x = np.arange(1,N+1)
        np.random.shuffle(x)

        # Loss records
        l = loss(x)
        L  = np.array([l])
        for k in range(1, MAX_ITERS):    

            i, j = idx_pairs[np.random.choice(C, size=1)][0]
            diff = loss_diff(x,i,j)
            acc = min(1, np.exp(-B[t]*diff))
            if np.random.rand() < acc:
                x = swap(x,i,j)
                l += diff

            if (k > THRESH):
                L = np.append(L, l)    

            if len(L) == M: break
        Z_ratios = np.append(Z_ratios, np.mean(np.exp(-(B[t+1] - B[t]) * L)))

    Z_inf_ratio = np.prod(Z_ratios)
    num_solutions = Z_inf_ratio * np.math.factorial(N)

18


In [332]:
print("estimated # of solutions (log method) = ", num_solutions)
print("Magnitude = ", np.log10(num_solutions))

estimated # of solutions (log method) =  8.649962751386016e+21
Magnitude =  21.937014237302694


In [264]:
np.log10(234_907_967_154_122_528)

17.370897746587715